In [36]:
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

# text processing libraries
import re
import string
import nltk
from nltk.corpus import stopwords

In [33]:
df = pd.read_csv('mtsamples.csv')

In [35]:
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [38]:
df['medical_specialty'].value_counts()

 Surgery                          1103
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        372
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  230
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Obstetrics / Gynecology           160
 Urology                           158
 Discharge Summary                 108
 ENT - Otolaryngology               98
 Neurosurgery                       94
 Hematology - Oncology              90
 Ophthalmology                      83
 Nephrology                         81
 Emergency Room Reports             75
 Pediatrics - Neonatal              70
 Pain Management                    62
 Psychiatry / Psychology            53
 Office Notes                       51
 Podiatry                           47
 Dermatology                        29
 Cosmetic / Plastic Surgery         27
 Dentistry               

In [39]:
possible_labels = df['medical_specialty'].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{' Allergy / Immunology': 0,
 ' Bariatrics': 1,
 ' Cardiovascular / Pulmonary': 2,
 ' Neurology': 3,
 ' Dentistry': 4,
 ' Urology': 5,
 ' General Medicine': 6,
 ' Surgery': 7,
 ' Speech - Language': 8,
 ' SOAP / Chart / Progress Notes': 9,
 ' Sleep Medicine': 10,
 ' Rheumatology': 11,
 ' Radiology': 12,
 ' Psychiatry / Psychology': 13,
 ' Podiatry': 14,
 ' Physical Medicine - Rehab': 15,
 ' Pediatrics - Neonatal': 16,
 ' Pain Management': 17,
 ' Orthopedic': 18,
 ' Ophthalmology': 19,
 ' Office Notes': 20,
 ' Obstetrics / Gynecology': 21,
 ' Neurosurgery': 22,
 ' Nephrology': 23,
 ' Letters': 24,
 ' Lab Medicine - Pathology': 25,
 ' IME-QME-Work Comp etc.': 26,
 ' Hospice - Palliative Care': 27,
 ' Hematology - Oncology': 28,
 ' Gastroenterology': 29,
 ' ENT - Otolaryngology': 30,
 ' Endocrinology': 31,
 ' Emergency Room Reports': 32,
 ' Discharge Summary': 33,
 ' Diets and Nutritions': 34,
 ' Dermatology': 35,
 ' Cosmetic / Plastic Surgery': 36,
 ' Consult - History and Phy.': 37,
 ' 

In [40]:
df['label'] = df['medical_specialty'].replace(label_dict)

In [41]:
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords,label
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller...",0
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh...",1
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart...",1
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple...",2
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo...",2


In [42]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values,
                                                  df.label.values,
                                                  test_size=0.15,
                                                  random_state=42,
                                                  stratify=df.label.values)

In [43]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [44]:
df.groupby(['medical_specialty', 'label', 'data_type']).count()

Unnamed: 0  description  sample_name  \
medical_specialty     label data_type                                         
 Allergy / Immunology 0     train               6            6            6   
                            val                 1            1            1   
 Autopsy              39    train               7            7            7   
                            val                 1            1            1   
 Bariatrics           1     train              15           15           15   
...                                           ...          ...          ...   
 Speech - Language    8     val                 1            1            1   
 Surgery              7     train             937          937          937   
                            val               166          166          166   
 Urology              5     train             134          134          134   
                            val                24           24           24   

                                       transcription  keywords  
medical_specialty     label data_type                           
 Allergy / Immunology 0     train                  6         3  
                            val                    1         0  
 Autopsy              39    train                  7         0  
                            val                    1         0  
 Bariatrics           1     train                 15        15  
...                                              ...       ...  
 Speech - Language    8     val                    1         1  
 Surgery              7     train                924       881  
                            val                  164       155  
 Urology              5     train                132       120  
                            val                   24        22  

[80 rows x 5 columns]

In [45]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

In [75]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


def text_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    #remove_stopwords = [w for w in tokenized_text if w not in stopwords.words('english')]
    combined_text = ' '.join(tokenized_text)
    return combined_text

df['transcription']=df['transcription'].apply(str).apply(lambda x: text_preprocessing(x))
df['medical_specialty']=df['medical_specialty'].apply(str).apply(lambda x: text_preprocessing(x))
trainlist=df[df.data_type=='train'].transcription.values.tolist()
vallist=df[df.data_type=='val'].transcription.values.tolist()

In [76]:
encoded_data_train = tokenizer.batch_encode_plus(
    trainlist,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    vallist,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [77]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [78]:
len(dataset_train), len(dataset_val)

(4220, 746)

In [79]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [80]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val,
                                   sampler=SequentialSampler(dataset_val),
                                   batch_size=batch_size)

In [81]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5,
                  eps=1e-8)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [82]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [83]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [84]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [85]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [86]:
def evaluate(dataloader_val):

    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total/len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [87]:
for epoch in tqdm(range(1, epochs+1)):

    model.train()

    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()

        batch = tuple(b.to(device) for b in batch)

        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        outputs = model(**inputs)

        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})


    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')

    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')

    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1407 [00:00<?, ?it/s]


Epoch 1
Training loss: 2.66251431357886
Validation loss: 2.3076506099068976
F1 Score (Weighted): 0.26527021753588326


Epoch 2:   0%|          | 0/1407 [00:00<?, ?it/s]


Epoch 2
Training loss: 2.144982382009109
Validation loss: 2.049211314404346
F1 Score (Weighted): 0.2668219847855345


...


Epoch 3
Training loss: 1.8829245648234976
Validation loss: 1.941953841701569
F1 Score (Weighted): 0.23744804102835987


Epoch 4:   0%|          | 0/1407 [00:00<?, ?it/s]


Epoch 4
Training loss: 1.7286217097449827
Validation loss: 1.9265217043788558
F1 Score (Weighted): 0.2503609218979429


Epoch 5:   0%|          | 0/1407 [00:00<?, ?it/s]


Epoch 5
Training loss: 1.630539761465665
Validation loss: 1.9200260620040588
F1 Score (Weighted): 0.23061532089105896


In [88]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [89]:
model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [90]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [91]:
accuracy_per_class(predictions, true_vals)

Class:  Allergy / Immunology
Accuracy: 0/1

Class:  Bariatrics
Accuracy: 0/3

Class:  Cardiovascular / Pulmonary
Accuracy: 38/56

Class:  Neurology
Accuracy: 5/33

Class:  Dentistry
Accuracy: 0/4

Class:  Urology
Accuracy: 1/24

Class:  General Medicine
Accuracy: 0/39

Class:  Surgery
Accuracy: 111/164

Class:  Speech - Language
Accuracy: 0/1

Class:  SOAP / Chart / Progress Notes
Accuracy: 0/25

Class:  Sleep Medicine
Accuracy: 0/3

Class:  Rheumatology
Accuracy: 0/2

Class:  Radiology
Accuracy: 20/41

Class:  Psychiatry / Psychology
Accuracy: 0/8

Class:  Podiatry
Accuracy: 0/7

Class:  Physical Medicine - Rehab
Accuracy: 0/3

Class:  Pediatrics - Neonatal
Accuracy: 0/11

Class:  Pain Management
Accuracy: 0/8

Class:  Orthopedic
Accuracy: 36/53

Class:  Ophthalmology
Accuracy: 0/12

Class:  Office Notes
Accuracy: 0/8

Class:  Obstetrics / Gynecology
Accuracy: 0/24

Class:  Neurosurgery
Accuracy: 0/14

Class:  Nephrology
Accuracy: 0/12

Class:  Letters
Accuracy: 0/3

Class:  Lab Medic